# Laboratorio: Analisi Flussi di Popolazione con pandas

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Oggetto: Analisi Flussi tra Stati USA

- L'ufficio del censimento degli Stati Uniti d'America pubblica annualmente un riepilogo dei flussi della popolazione tra Stati
- I dataset per ogni anno in formato XLS (Excel) possono essere reperiti alla pagina https://www.census.gov/data/tables/time-series/demo/geographic-mobility/state-to-state-migration.html
- In questa esercitazione, analizziamo i flussi interni tra gli Stati nel 2016 e analizziamo in base ad essi quali Stati possano essere considerati più simili tra loro
- Eseguire la seguente cella per scaricare il file con i dati dell'esercitazione, se non presente

In [1]:
import os.path
if not os.path.exists("usa_census.npz"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/vxh8Y", "usa_census.npz")

## Caricamento dei Dati

- Abbiamo raccolto i dati da analizzare in un _archivio NumPy_, un file contenente un insieme di array con nomi associati
- Scarichiamo il file e carichiamo da esso i dati, usiamo la funzione `load` di NumPy

In [2]:
import numpy as np
data = np.load("usa_census.npz")

- L'oggetto `data` ottenuto è simile ad un dizionario, i cui valori sono le matrici e le cui chiavi sono i loro nomi
- Otteniamo una lista dei nomi delle matrici caricate

In [3]:
print(", ".join(data.keys()))

states, population, area, same_house, same_state, other_state, state_to_state, from_abroad


## Dati Disponibili

I dati contenuti nel file sono i seguenti:
- `states`: vettore con i nomi degli Stati
- `population`: vettore con la popolazione totale attuale di ogni Stato
- `area`: vettore con la superficie di terra di ciascuno stato in miglia quadrate
- `same_house`: vettore con numero di abitanti per Stato che non hanno cambiato residenza nell'ultimo anno
- `same_state`: vettore con numero di abitanti per Stato che hanno cambiato residenza nell'ultimo anno, ma non lo Stato
- `other_state`: vettore con numero di abitanti per Stato che si sono trasferiti da un altro Stato nell'ultimo anno
- `state_to_state`: matrice col numero di abitanti trasferitisi nell'ultimo anno da ciascuno Stato ad un altro
- `from_abroad`: vettore con numero di abitanti per Stato trasferitisi negli USA dall'estero nell'ultimo anno

A ciascuno Stato corrisponde un indice in base all'ordine alfabetico: in ciascun vettore, il valore relativo a quello Stato si trova allo stesso indice

- Possiamo verificare tipo e forma di ciascun array

In [4]:
for name, array in data.items():
    print("{:>15s}: {:>8s} {}".format(name, str(array.dtype), array.shape))

         states:   object (51,)
     population:    int64 (51,)
           area:  float64 (51,)
     same_house:    int64 (51,)
     same_state:    int64 (51,)
    other_state:    int64 (51,)
 state_to_state:    int64 (51, 51)
    from_abroad:    int64 (51,)


- Vediamo ad esempio l'array con i nomi degli stati

In [5]:
data["states"]

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

- Per comodità, eseguire il seguente codice per importare automaticamente tutti gli elementi di `data` come variabili locali
  - la funzione `exec` esegue codice Python dato in una stringa **(può eseguire codice arbitrario, va usata con cautela!)**

In [6]:
for key, value in data.items():
    exec(key + " = value")

- In questo modo potremo usare ad es. `states` al posto di `data["states"]`

## Estrarre Informazioni dagli Array

- Per estrarre informazioni su Stati specifici, dobbiamo conoscere la posizione di ciascuno Stato negli array
- Ad esempio, sapendo che California è il 5° stato in ordine alfabetico, per conoscerne il numero di abitanti posso scrivere:

In [7]:
population[4]

38783436

- Se non conoscessi la posizione, dovrei individuarla dall'array `states` con i nomi
- Ad esempio, usando l'indicizzazione binaria per individuare lo stato col nome corretto:

In [8]:
population[states == "California"]

array([38783436])

## pandas

- **pandas** è una libreria Python di uso comune per lavorare con dati in forma tabulare
- Offre strutture dati simili agli array, ma dotate di _indici_ che etichettano i dati al loro interno
- Questo rende semplice reperire dati specifici, ad esempio la popolazione di uno stato specifico come da esempio sopra
- Iniziamo importando il package `pandas` con l'alias convenzionale `pd`

In [9]:
import pandas as pd

## Serie

- Una **_serie_** pandas è un vettore di valori simile ad un array 1D NumPy con un'etichetta associata a ciascuno
- La sequenza di etichette costituisce l'_indice_ della serie, anch'esso un array 1D

## Da Array a Serie

- Il costruttore `Series` permette di creare una serie di cui sono forniti i dati e l'indice (`index`) in due vettori di uguale lunghezza
- Creiamo la serie `population` che sostituisca l'array con lo stesso nome
  - i dati della serie sono contenuti nell'array stesso
  - le etichette sono i nomi degli stati contenuti nell'array `states`

In [10]:
population = pd.Series(population, index=states)

- Visualizziamo la serie, utilizzando il metodo `head` per estrarne solo i primi N elementi

In [11]:
population.head(7)

Alabama         4810126
Alaska           731760
Arizona         6851836
Arkansas        2949650
California     38783436
Colorado        5476928
Connecticut     3541758
dtype: int64

- Sulla sinistra vediamo le etichette (nomi degli Stati), sulla destra i valori (popolazione di ciascuno Stato)

## Attributi di Base di una Serie

- Una serie è composta dai dati e dalle etichette: gli attributi `values` e `index` restituiscono questi elementi in forma di vettori

In [12]:
population.values   [:10]

array([ 4810126,   731760,  6851836,  2949650, 38783436,  5476928,
        3541758,   942073,   672022, 20401575])

In [13]:
population.index    [:10]

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'District of Columbia', 'Florida'],
      dtype='object')

- Il numero di elementi totale si può ottenere con la funzione `len` di Python o (per analogia con gli array) con l'attributo `size`

In [14]:
len(population)

51

In [15]:
population.size

51

## Selezionare Dati da una Serie

- Le serie supportano in pratica le stesse tecniche di selezione degli array, ma usando le **etichette al posto degli indici numerici**
- Per cui, per ottenere la popolazione della California, si scrive semplicemente:

In [16]:
population["California"]

38783436

- Se si seleziona un'etichetta inesistente, si ha un `KeyError` come per i dizionari `{...}`

- Si può selezionare un intervallo tra due etichette (sono inclusi entrambi gli estremi)

In [17]:
population["Arizona":"Colorado"]

Arizona        6851836
Arkansas       2949650
California    38783436
Colorado       5476928
dtype: int64

- Si può selezionare un sottoinsieme della serie indicando una lista di etichette

In [18]:
# popolazione Stati della costa ovest
population[["Washington", "Oregon", "California"]]

Washington     7202119
Oregon         4052221
California    38783436
dtype: int64

## Creazione delle altre Serie

- Creiamo una serie per ciascuno degli altri vettori di dati, utilizzando sempre il vettore di nomi degli Stati `states` come indice
  - _(gli array originali rimangono disponibili nell'oggetto `data`)_

In [19]:
area = pd.Series(area, index=states)
same_house = pd.Series(same_house, index=states)
same_state = pd.Series(same_state, index=states)
other_state = pd.Series(other_state, index=states)
from_abroad = pd.Series(from_abroad, index=states)

## Operazioni tra Serie

- Come per gli array, è possibile effettuare operazioni elemento per elemento tra due serie o tra una serie ed un valore singolo
- Si possono utilizzare operazioni binarie (`+`, `*`, ...) e funzioni universali di NumPy (`np.log`, ...)
- Nel caso di due serie, le operazioni sono applicate tra **elementi di uguale etichetta** (indipendentemente dalla posizione)
- Ad esempio, per convertire la superficie da miglia quadrate a chilometri quadrati (1 mi² = 2,59 km²):

In [20]:
area_km2 = area * 2.59
# stampo i primi 3 elementi
area_km2.head(3)

Alabama    1.311714e+05
Alaska     1.477960e+06
Arizona    2.942087e+05
dtype: float64

- Per ottenere la densità di popolazione di ciascuno Stato (in abitanti per km²):

In [21]:
density = population / area_km2
density.head(3)

Alabama    36.670538
Alaska      0.495115
Arizona    23.289035
dtype: float64

- Per ottenere il logaritmo in base 10 della popolazione (utilizzabile ad es. per creare un grafico in tale scala):

In [22]:
np.log10(population)   .head(3)

Alabama    6.682156
Alaska     5.864369
Arizona    6.835807
dtype: float64

## Serie Booleane e Selezione per Condizioni

- Applicando una comparazione ad una serie, si ottiene una serie di valori booleani
- Ad esempio, per ottenere la serie che associa `True` agli Stati non più grandi di 5.000 miglia quadrate:

In [23]:
small_states = area <= 5000
small_states.head(10)

Alabama                 False
Alaska                  False
Arizona                 False
Arkansas                False
California              False
Colorado                False
Connecticut              True
Delaware                 True
District of Columbia     True
Florida                 False
dtype: bool

- Come per gli array, una serie booleana può essere usata per selezionare solamente alcuni elementi di una serie di dati
  - sono selezionati solamente gli elementi alla cui etichetta nella serie booleana è associato `True`
- Ad esempio, per mostrare l'area solamente degli Stati piccoli individuati sopra:

In [24]:
area[area <= 5000]
# o anche: area[small_states]

Connecticut             4842.36
Delaware                1948.54
District of Columbia      61.05
Rhode Island            1033.81
dtype: float64

- La serie da cui sono selezionati i dati e quella usata come condizione possono anche differire...

In [25]:
population[area <= 5000]

Connecticut             3541758
Delaware                 942073
District of Columbia     672022
Rhode Island            1045691
dtype: int64

- Possiamo combinare serie booleane con gli operatori `&` (AND), `|` (OR), `^` (XOR), `~` (NOT)
  - tali operatori hanno normalmente priorità superiore alle comparazioni (`>`, `<=`, ...), usare le parentesi per evitare errori

In [26]:
population[(population >= 1_000_000) & (area <= 5000)]

Connecticut     3541758
Rhode Island    1045691
dtype: int64

## Statistiche sulle Serie

- Le serie offrono metodi simili a quelli degli array per aggregare i dati: `sum`, `mean`, `min`, `max`, ...
- Ad esempio, per ottenere la popolazione totale di tutti gli Stati:

In [27]:
population.sum()

319361956

- Per ottenere la popolazione nello Stato dove è maggiore:

In [28]:
population.max()

38783436

- Per conoscere a quale Stato corrisponde, uso il metodo `idxmax` che restituisce l'etichetta del valore maggiore

In [29]:
population.idxmax()

'California'

## Esercizi A: Serie

- **1)** Qual è la densità di popolazione dello Stato più più piccolo?
- **2)** Quanti sono gli Stati la cui popolazione è superiore al milione di abitanti?
  - ricordare che `True` = 1 e `False` = 0
- **3)** Qual è il totale della popolazione degli Stati sulla costa ovest (Washington, Oregon e California)?
- **4)** Qual è la densità media degli Stati con almeno 10 milioni di abitanti?

## DataFrame

- Un `DataFrame` pandas contiene dati strutturati in forma tabulare, dove in genere
  - ogni riga della tabella rappresenta un elemento dell'insieme che si sta analizzando (una persona, un territorio, ...)
  - ogni colonna rappresenta una caratteristica di interesse (_feature_) degli elementi (età, popolazione, ...)
- Un DataFrame è assimilabile ad una serie, ma in due dimensioni
  - possiede molti metodi presenti anche nelle serie e negli array

## Creare un DataFrame

- Per creare un DataFrame possiamo passare un dizionario con una serie di valori per ciascuna colonna, la cui chiave è il nome
  - l'indice delle serie, identico per tutte, è usato come indice delle righe del DataFrame

In [30]:
census = pd.DataFrame({
    "population": population,
    "from_abroad": from_abroad,
    "area": area_km2
})

- Visualizziamo le prime righe del DataFrame come esempio...

In [31]:
census.head(5)

population  from_abroad          area
Alabama        4810126        16062  1.311714e+05
Alaska          731760         6559  1.477960e+06
Arizona        6851836        53749  2.942087e+05
Arkansas       2949650         9051  1.347719e+05
California    38783436       336614  4.034682e+05

- In alto in grassetto sono scritti i nomi delle colonne, che costituiscono _l'indice delle colonne_
- A sinistra sempre in grassetto sono scritti i nomi degli Stati, che costituiscono _l'indice delle righe_
  - l'indice delle righe **non** conta come colonna

- Un modo alternativo per creare un DataFrame è passare una matrice (array 2D) con i dati e vettori con nomi di righe e colonne
- Ad esempio, possiamo convertire in DataFrame la matrice `state_to_state` che indica il numero di persone trasferitesi da uno Stato all'altro nell'ultimo anno
- Creiamo il DataFrame passando i dati nella matrice e usando il vettore dei nomi `states` come indice sia delle righe (`index`) che delle colonne (`columns`)

In [32]:
state_to_state = pd.DataFrame(state_to_state, index=states, columns=states)

- Abbiamo così convertito la matrice in una tabella leggibile, di cui quì visualizziamo una parte

In [33]:
state_to_state.iloc[:7, :5]

Alabama  Alaska  Arizona  Arkansas  California
Alabama            0     576     1022       495        6611
Alaska           423       0     1176        65        3593
Arizona          894    1946        0      1205       64756
Arkansas        2057     103      836         0        4026
California      3045    4206    33757      4282           0
Colorado        2328    1698    13015       808       26909
Connecticut     1102     331      309       960        3979

- Ogni riga rappresenta lo stato di destinazione, ogni colonna lo stato d'origine
  - ad esempio, 423 persone si sono trasferite dall'Alabama all'Alaska

## Estrarre ed Aggiungere Colonne

- Il DataFrame è utilizzabile come un dizionario i cui valori sono le colonne e le chiavi i loro nomi
- Selezionando una chiave, viene restituita la colonna con quel nome in forma di serie
  - l'indice della serie restituita è l'indice del DataFrame

In [34]:
census["population"]   .head(3)

Alabama    4810126
Alaska      731760
Arizona    6851836
Name: population, dtype: int64

- Seguendo la stessa logica, è possibile aggiungere colonne assegnando una serie di valori ad un nome
  - se una colonna col nome dato esiste già, viene sovrascritta
  - se viene passata una serie, i valori sono assegnati per corrispondenza tra etichette e indice delle righe
- Questo permette di creare facilmente colonne con valori derivati dalle altre
- Ad esempio, per aggiungere una colonna con la densità di popolazione _(senza usare la serie creata in precedenza)_:

In [35]:
census["density"] = census["population"] / census["area"]

- La colonna viene così aggiunta a destra nel DataFrame:

In [36]:
census.head(5)

population  from_abroad          area    density
Alabama        4810126        16062  1.311714e+05  36.670538
Alaska          731760         6559  1.477960e+06   0.495115
Arizona        6851836        53749  2.942087e+05  23.289035
Arkansas       2949650         9051  1.347719e+05  21.886240
California    38783436       336614  4.034682e+05  96.125142

## Statistiche sui Dati

- Anche sui DataFrame sono disponibili i metodi `sum`, `mean`, `min`, ... per aggregare i dati
- Di default, restituiscono una serie con la statistica calcolata colonna per colonna

In [37]:
# medie per Stato
census.mean()

population     6.261999e+06
from_abroad    4.307184e+04
area           1.793654e+05
density        1.585860e+02
dtype: float64

In [38]:
# somme per ciascuno Stato
census.sum()

population     3.193620e+08
from_abroad    2.196664e+06
area           9.147635e+06
density        8.087884e+03
dtype: float64

- Si noti che non sempre tutte le statistiche che si possono estrarre sono significative: ad esempio la somma ha senso sulla popolazione e sull'area, ma non sulla densità

- Il metodo `describe` fornisce rapidamente un'insieme di statistiche sui valori di ciascuna colonna, utili ad analizzarne la distribuzione

In [39]:
census.describe()

population    from_abroad          area      density
count  5.100000e+01      51.000000  5.100000e+01    51.000000
mean   6.261999e+06   43071.843137  1.793654e+05   158.585968
std    7.156688e+06   65074.514352  2.215125e+05   592.990817
min    5.775670e+05    2105.000000  1.581195e+02     0.495115
25%    1.737232e+06    8790.000000  8.633639e+04    18.459321
50%    4.385213e+06   19460.000000  1.388881e+05    40.722584
75%    7.026978e+06   52012.000000  2.089942e+05    86.390197
max    3.878344e+07  336614.000000  1.477960e+06  4250.089331

La tabella ottenuta mostra:
- `count` = valori non mancanti (51 in tutte e tre le colonne)
- `mean` = media (ad es. la popolazione media per stato è di 6,26 milioni di abitanti)
- `std` = deviazione standard
- `min`/`max` = valori minimo/massimo
- `25%`/`50%`/`75%` = percentili (ad es. il 25% degli stati ha densità di popolazione inferiore a 18,46 abitanti per km²)

- Normalmente le statistiche sono calcolate per colonne (riducendo le righe), in quanto si tratta dell'esigenza più comune
  - nella tabella `census`, così come in tanti altri casi pratici, ogni colonna ha valori in scale diverse, non ha senso calcolare somma, media, ecc. per riga
  - `sum()` in un DataFrame corrisponde in pratica a `sum(0)` in una matrice NumPy
- Nel caso si vogliano calcolare le statistiche per righe (riducendo le colonne), si può specificare `axis=1`
  - questo corrisponde in pratica a `sum(1)` su una matrice NumPy

- Ad esempio, applicando `sum()` in modo standard al DataFrame `state_to_state`, otteniamo il totale di abitanti trasferitisi **da** ogni stato (colonne)...

In [40]:
state_to_state.sum()   .head(3)

Alabama     99892
Alaska      42074
Arizona    192103
dtype: int64

- ...mentre applicando `sum(axis=1)` otteniamo il totale di abitanti trasferitisi **verso** ogni stato (righe)...

In [41]:
state_to_state.sum(axis=1)   .head(3)

Alabama    122220
Alaska      31300
Arizona    273257
dtype: int64

- ...che è esattamente quanto riportato nella serie `other_state`: possiamo verificarlo col metodo `equals` che indica se due serie sono identiche

In [42]:
state_to_state.sum(axis=1).equals(other_state)

True

## Selezione

- Per selezionare una porzione di DataFrame, vanno indicate le righe che le colonne da includere
- Esistono diversi _selettori_, che consentono di selezionare parti di un DataFrame `X` in modo diverso:
  - `X.loc[...]` seleziona righe e colonne specificate **per etichetta** (es.: colonna "population")
  - `X.iloc[...]` seleziona righe e colonne specificate **per posizione** (es.: colonna 0)
- In entrambi i casi vanno specificate righe e colonne da selezionare come avviene nelle matrici
  - si può selezionare per valore singolo, intervallo, lista di valori o serie booleana
  - `:` indica di selezionare tutte le righe o le colonne, se usato per le colonne può essere omesso

- Ad esempio, abbiamo già visto sopra come selezionare solamente le prime righe e colonne di un DataFrame:

In [43]:
state_to_state.iloc[:3, :5]

Alabama  Alaska  Arizona  Arkansas  California
Alabama        0     576     1022       495        6611
Alaska       423       0     1176        65        3593
Arizona      894    1946        0      1205       64756

- Selezionando una singola riga di un DataFrame, otteniamo una serie con i valori di tutte le colonne

In [44]:
census.loc["California", :]   # ", :" può essere omesso

population     3.878344e+07
from_abroad    3.366140e+05
area           4.034682e+05
density        9.612514e+01
Name: California, dtype: float64

- Usando serie booleane, possiamo selezionare righe (o colonne) per condizioni

In [45]:
# righe relative agli Stati con meno di 700.000 abitanti
census.loc[census["population"] < 700000]

population  from_abroad         area      density
District of Columbia      672022        11155     158.1195  4250.089331
Vermont                   619387         2113   23871.1494    25.947096
Wyoming                   577567         2105  251471.2326     2.296752

In [46]:
# sole densità di popolazione relative a Stati con superficie inferiore di 10.000 km²
census.loc[census["area"] < 10000, "density"]

Delaware                 186.670404
District of Columbia    4250.089331
Rhode Island             390.537622
Name: density, dtype: float64

## Ordinamento

- La funzione `sort_values` restituisce una copia del DataFrame con le righe ordinate secondo i valori di una o più colonne
- Possiamo usarla ad esempio per visualizzare i 5 Stati più popolati
  - specifichiamo `ascending=False` per ottenere un ordinamento decrescente

In [47]:
census.sort_values("population", ascending=False).head(5)

population  from_abroad         area     density
California    38783436       336614  403468.1798   96.125142
Texas         27472626       234749  676590.1289   40.604533
Florida       20401575       232838  138888.1284  146.892144
New York      19526443       166069  122057.3760  159.977575
Illinois      12654142        65886  143794.0287   88.001860

## Esercizi B: DataFrame

Estrarre le risposte dai DataFrame, estraendo colonne da di esse dove necessario. Non usare le serie create in precedenza.

- **1)** Qual è la superficie della California?
- **2)** Qual è la popolazione (colonna 0) del 13° Stato nella tabella?
- **3)** Qual è la densità di popolazione dello Stato più piccolo
- **4)** Qual è lo Stato da cui sono emigrate più persone nello scorso anno verso altri Stati?
- **5)** Qual è la popolazione media degli Stati con almeno l'1% di popolazione immigrato dall'estero nell'ultimo anno?